## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Étiquetage morphosyntaxique (POS Tagging) 
- Lemmatisation
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de collocations significatives (en fonction du Log-likelihood ratio)
- Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt
- Extraction de termes MeSH et SNOMED présents dans les données

Ajouts (2022-07-12)
- LLR (semble fonctionner maintenant)

### **Lire le corpus** 

In [351]:
import shutil, re, pandas, random
from os import listdir, chdir, path
from pathlib import Path

acteur = 'pinel'
sous_corpus = False 
tag = ''

# Change the directory
if sous_corpus:
    base_path = '../03-corpus/2-sous-corpus/'
    file_path = path.join(base_path, acteur, tag)

else: 
    base_path = '../03-corpus/2-data/1-fr/'
    file_path = path.join(base_path, acteur) + '.csv'

In [352]:
from pandas import *
with open(file_path, "r", encoding = "UTF-8") as f:
        data = read_csv(file_path)
        text = data['text'].tolist()

In [353]:
nb_docs = len(text)

print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 523 documents.


### **Extraire un échantillon aléatoire**

Sinon, on n'arrive pas à traiter la totalité du corpus pour des raisons de performance

In [354]:
n = round(1 * nb_docs)
corpus = random.sample(text, n)

print("On va travailler sur un échantillon correspondant à environ 40 % des documents du corpus, soit {} documents". format(len(corpus)))

corpus = " ".join([(re.sub('\d', '', t.strip('\n').lower().replace('’', '\''))) for t in text])

On va travailler sur un échantillon correspondant à environ 40 % des documents du corpus, soit 523 documents


**NLTK**\
https://www.nltk.org/ 

In [355]:
import nltk
#nltk.download(['popular'])

In [356]:
punct = '!#$%&()*+,-/:;<=>?@[\]^_{|}~©'

for t in punct:
    corpus = corpus.replace(t, ' ').replace("  ", " ")

### **Filtrage (MWE - stopwords formés de plusieurs tokens)**
Surtout pour filtrer les expressions relatives à l'architecture d'information / navigation Web

In [357]:
file_path = '../04-filtrage/mwe_stopwords.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mwe_sw = [t.lower().strip('\n') for t in f.readlines()]

In [358]:
for mwe in mwe_sw:
    corpus = corpus.replace(mwe, '')

### **Tokenisation / POS tagging** (TreeTagger)  
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [359]:
from nltk.tokenize import RegexpTokenizer

# Seulement les caractères alphabétiques
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")

tokens = tokenizer_re.tokenize(corpus)
len_corpus = len(tokens)

print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len_corpus))

Avec le RegExpTokenizer, notre corpus contient 788370 tokens.


In [360]:
tokens[:50]

['institut',
 'national',
 'de',
 'psychiatrie',
 'légale',
 'philippe',
 'pinel',
 'a',
 'a',
 'a',
 'événements',
 'nouvelles',
 'covid',
 'carrières',
 'forensia',
 'fondation',
 'soins',
 'aux',
 'patients',
 'familles',
 'et',
 'visiteurs',
 'services',
 'aux',
 'professionnels',
 'recherche',
 'volet',
 'académique',
 "l'",
 'institut',
 'menu',
 'soins',
 'aux',
 'patients',
 'voir',
 'tout',
 'philosophie',
 'des',
 'soins',
 'expertise',
 'sécurité',
 'des',
 'soins',
 'équipes',
 'éthique',
 'intégration',
 'sociale',
 'programmes',
 'et',
 'traitements']

In [361]:
corpus = " ".join(tokens).replace("' ", "'")

In [362]:
import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')

### **Mapping POS Tags** (FRMG)

Pour utiliser adéquatement notre lemmatiseur par la suite (FrenchLefffLemmatizer), on va mapper les étiquettes morphosyntaxiques du TreeTagger à celles que prend le lemmatiseur (celles issues de FRMG)

http://alpage.inria.fr/frmgwiki/content/tagset-frmg

In [363]:
file_path = '../04-filtrage/mapping_treeTagger_lefff.csv'

with open(file_path) as f:
    csv = read_csv(f)

treeTag = [term for term in csv['TreeTagger'].tolist()] 
lefff = [term for term in csv['Lefff'].tolist()]

mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

In [364]:
# tagged = [[t.split('\t')[0], mapping[t.split('\t')[1]]] for t in tagger.tag_text(corpus)]
tagged = [(t.split('\t')[0], mapping[t.split('\t')[1]]) for t in tagger.tag_text(corpus)]


#if len(t.split('\t')) >1

In [365]:
tagged

[('institut', 'nc'),
 ('national', 'adj'),
 ('de', 'prep'),
 ('psychiatrie', 'nc'),
 ('légale', 'adj'),
 ('philippe', 'nc'),
 ('pinel', 'adj'),
 ('a', 'v'),
 ('a', 'v'),
 ('a', 'v'),
 ('événements', 'nc'),
 ('nouvelles', 'adj'),
 ('covid', 'adj'),
 ('carrières', 'nc'),
 ('forensia', 'v'),
 ('fondation', 'nc'),
 ('soins', 'nc'),
 ('aux', 'prep'),
 ('patients', 'adj'),
 ('familles', 'nc'),
 ('et', 'csu'),
 ('visiteurs', 'nc'),
 ('services', 'nc'),
 ('aux', 'prep'),
 ('professionnels', 'adj'),
 ('recherche', 'nc'),
 ('volet', 'nc'),
 ('académique', 'adj'),
 ("l'", 'det'),
 ('institut', 'nc'),
 ('menu', 'adj'),
 ('soins', 'nc'),
 ('aux', 'prep'),
 ('patients', 'nc'),
 ('voir', 'v'),
 ('tout', 'pro'),
 ('philosophie', 'nc'),
 ('des', 'prep'),
 ('soins', 'nc'),
 ('expertise', 'v'),
 ('sécurité', 'nc'),
 ('des', 'prep'),
 ('soins', 'nc'),
 ('équipes', 'nc'),
 ('éthique', 'adj'),
 ('intégration', 'nc'),
 ('sociale', 'adj'),
 ('programmes', 'nc'),
 ('et', 'csu'),
 ('traitements', 'nc'),
 ('prog

### **Lemmatisation** (FrenchLefffLemmatizer)

https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer

In [366]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

In [367]:
lemmatizer = FrenchLefffLemmatizer()

In [368]:
lemmas = []
for term in tagged:
    term_l = []
    if lemmatizer.lemmatize(term[0], term[1]) == []:
        term_l = (lemmatizer.lemmatize(term[0]), term[1])
    
    elif type(lemmatizer.lemmatize(term[0], term[1])) == str:
        term_l  = (lemmatizer.lemmatize(term[0], term[1]), term[1])

    else:
        term_l = tuple(lemmatizer.lemmatize(term[0], term[1])[0])
    
    lemmas.append(term_l)

In [369]:
lemmas

[('institut', 'nc'),
 ('national', 'adj'),
 ('de', 'prep'),
 ('psychiatrie', 'nc'),
 ('légal', 'adj'),
 ('philippe', 'nc'),
 ('pinel', 'adj'),
 ('avoir', 'v'),
 ('avoir', 'v'),
 ('avoir', 'v'),
 ('événement', 'nc'),
 ('nouveau', 'adj'),
 ('covid', 'adj'),
 ('carrière', 'nc'),
 ('forensia', 'v'),
 ('fondation', 'nc'),
 ('soin', 'nc'),
 ('aux', 'prep'),
 ('patient', 'adj'),
 ('famille', 'nc'),
 ('et', 'csu'),
 ('visiteur', 'nc'),
 ('service', 'nc'),
 ('aux', 'prep'),
 ('professionnel', 'adj'),
 ('recherche', 'nc'),
 ('volet', 'nc'),
 ('académique', 'adj'),
 ('le', 'det'),
 ('institut', 'nc'),
 ('menu', 'adj'),
 ('soin', 'nc'),
 ('aux', 'prep'),
 ('patient', 'nc'),
 ('voir', 'v'),
 ('tout', 'pro'),
 ('philosophie', 'nc'),
 ('des', 'prep'),
 ('soin', 'nc'),
 ('expertiser', 'v'),
 ('sécurité', 'nc'),
 ('des', 'prep'),
 ('soin', 'nc'),
 ('équipe', 'nc'),
 ('éthique', 'adj'),
 ('intégration', 'nc'),
 ('social', 'adj'),
 ('programme', 'nc'),
 ('et', 'csu'),
 ('traitement', 'nc'),
 ('programme'

### **Filtrage** (antidictionnaire)  

In [370]:
# Importer l'antidictionnaire pour filtrer les données

# Stopwords lemmatisés
file_path = '../04-filtrage/stopwords_lemmatized.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords_lemmatized = [w.strip('\n').lower() for w in f.readlines()]

# Stopwords fréquents en français (non lemmatisés)
file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais (non lemmatisés)
file_path = '../04-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords += [t.lower().strip('\n') for t in f.readlines()]

### **Collocations / Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook  

In [371]:
from nltk import everygrams
ngrammes = list(everygrams(tagged, min_len=2, max_len=5))
ngrammes_lemmatized = list(everygrams(lemmas, min_len=2, max_len=5))

In [372]:
print("Avant filtrage, on a {} ngrammes.".format(len(ngrammes)))

Avant filtrage, on a 3153430 ngrammes.


### **Filtrage (N-grammes)**

On retire les n-grammes qui apparaissent moins de 30 fois dans tout le corpus ou qui débutent ou terminent par :
- un stopword
- un mot de 1 lettre ou moins

Pour le reste du traitement, on arrête de considérer les frontières entre les phrases et entre les documents (nos ngrammes les respectent donc on n'en a plus besoin)

In [373]:
def extract_patterns(ngrammes):
    patterns = []

    for ng in ngrammes:
        phrase = []
        pattern = []
        for t in ng:
            phrase.append(t[0]) # token
            pattern.append(t[1]) # POS tag

        patterns.append([phrase, pattern])
        
    return patterns

phrases = extract_patterns(ngrammes)
phrases_lemmatized = extract_patterns(ngrammes_lemmatized)

In [374]:
phrases[:20]

[[['institut', 'national'], ['nc', 'adj']],
 [['institut', 'national', 'de'], ['nc', 'adj', 'prep']],
 [['institut', 'national', 'de', 'psychiatrie'], ['nc', 'adj', 'prep', 'nc']],
 [['institut', 'national', 'de', 'psychiatrie', 'légale'],
  ['nc', 'adj', 'prep', 'nc', 'adj']],
 [['national', 'de'], ['adj', 'prep']],
 [['national', 'de', 'psychiatrie'], ['adj', 'prep', 'nc']],
 [['national', 'de', 'psychiatrie', 'légale'], ['adj', 'prep', 'nc', 'adj']],
 [['national', 'de', 'psychiatrie', 'légale', 'philippe'],
  ['adj', 'prep', 'nc', 'adj', 'nc']],
 [['de', 'psychiatrie'], ['prep', 'nc']],
 [['de', 'psychiatrie', 'légale'], ['prep', 'nc', 'adj']],
 [['de', 'psychiatrie', 'légale', 'philippe'], ['prep', 'nc', 'adj', 'nc']],
 [['de', 'psychiatrie', 'légale', 'philippe', 'pinel'],
  ['prep', 'nc', 'adj', 'nc', 'adj']],
 [['psychiatrie', 'légale'], ['nc', 'adj']],
 [['psychiatrie', 'légale', 'philippe'], ['nc', 'adj', 'nc']],
 [['psychiatrie', 'légale', 'philippe', 'pinel'], ['nc', 'adj',

In [375]:
from nltk.probability import FreqDist

freq = FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases])
freq_lemmatized = FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases_lemmatized])

In [376]:
freq

FreqDist({'de la': 7682, 'psychiatrie légale': 7241, 'voir tout': 6204, 'à la': 6093, "de l'": 5913, "l'institut": 5906, 'en psychiatrie': 5505, 'en psychiatrie légale': 5452, 'pour les': 4438, 'centre de': 4337, ...})

In [377]:
def filtrer_phrases(phrases, freq):
        output = []
        for term in phrases:
                exp = " ".join(term[0]).replace("' ", "'")
                f = freq[exp]
                
                # f > x and - Si on veut filtrer par fréquence 
                if f > 20 and not term[0][0] in stopwords and len(term[0][0]) > 2 \
                and not term[0][-1] in stopwords and len(term[0][-1]) > 2 : 
                        pattern = " ".join(term[1])            
                        output.append([exp, pattern, f])  
                         
        return output

phrases = filtrer_phrases(phrases, freq)
phrases_lemmatized = filtrer_phrases(phrases_lemmatized, freq_lemmatized)

In [378]:
print("Après filtrage, on a {} occurrences de ngrammes.".format(len(phrases))) 

Après filtrage, on a 584857 occurrences de ngrammes.


In [379]:
def tabCSV(phrases, titre):
    base_path = '../04-filtrage/output/'
    tab = DataFrame(phrases, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates()
    tab.sort_values(["Fréquence"], 
                        axis=0,
                        ascending=[False], 
                        inplace=True)


    file_path = path.join(base_path, acteur, acteur)
    if sous_corpus:
       file_path = path.join(base_path, acteur, tag, tag)
    

    Path(file_path).mkdir(parents=True, exist_ok=True)

    tab.to_csv(file_path + titre)

    return tab.values.tolist()

In [380]:
phrases = tabCSV(phrases,'_n-grams.csv')
phrases_lemmatized = tabCSV(phrases_lemmatized, '_n-grams-lemmatized.csv')

In [381]:
print("Après filtrage, on a {} ngrammes uniques.".format(len(phrases)))

Après filtrage, on a 1702 ngrammes uniques.


### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

In [382]:
file_path = '../04-filtrage/mesh_patterns-fr.csv'

with open (file_path, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist()[:30] # Pour prendre seulement les 30 structures syntaxiques les plus fréquentes dans les MeSH

In [383]:
terms = [t for t in phrases if t[1] in patterns]
terms_lemmatized = [t for t in phrases_lemmatized if t[1] in patterns]

In [384]:
print("Le filtrage syntaxique élimine environ {} % des termes".format(round((len(phrases) - len(terms)) / len(phrases) * 100)))
print("On avait {} bigrammes, ".format(len(phrases)) + "on en a maintenant {}.".format(len(terms)))

Le filtrage syntaxique élimine environ 59 % des termes
On avait 1702 bigrammes, on en a maintenant 695.


In [385]:
import pandas as pd

def extract_terms(liste_terms, titre):
    file_path = '../04-filtrage/output/'
    tab = pd.DataFrame(terms, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates(subset='Expression', keep="last")
    tab.sort_values(["Fréquence"], 
                        axis=0,
                        ascending=[False], 
                        inplace=True)

    if sous_corpus:
        file_path = path.join(file_path, acteur, tag, tag)

    else :
        file_path = path.join(file_path, acteur, acteur)

                    
    tab.to_csv(file_path + titre)

extract_terms(terms, '_terms.csv')
extract_terms(terms_lemmatized, '_terms-lemmatized.csv')

### **Filtrage (Collocations statistiquement significatives)** Log-Likelihood Ratio

[Notebook - Collocation extraction methodologies compared](https://notebooks.githubusercontent.com/view/ipynb?azure_maps_enabled=false&browser=chrome&color_mode=auto&commit=33868e847376764d7733cd958986c88dedfaec97&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f746f64642d636f6f6b2f4d4c2d596f752d43616e2d5573652f333338363865383437333736373634643737333363643935383938366338386465646661656339372f70726f626162696c69737469635f6c616e67756167655f6d6f64656c696e672f636f6c6c6f636174696f6e5f65787472616374696f6e732e6970796e62&enterprise_enabled=false&logged_in=false&nwo=todd-cook%2FML-You-Can-Use&path=probabilistic_language_modeling%2Fcollocation_extractions.ipynb&platform=android&repository_id=167140788&repository_type=Repository&version=102)

On applique un test d'hypothèse statistique aux n-grammes sur lesquels une probabilité a été mesurée (Log-likelihood ratio) - seuls les n-grammes dont le test est significatif seront conservés.
On considère que l'apparition de ces collocations dans notre corpus n'est pas dûe au hasard.

In [386]:
def loglikelihood_ratio(c_prior, c_n, c_ngram, N):
    """
    Compute the ratio of two hypotheses of likelihood and return the ratio.
    The formula here and test verification values are taken from 
    Manning & Schūtze _Foundations of Statistical Natural Language Processing_ p.172-175
    Parameters:
    c_prior: count of word 1 if bigrams or count of [w1w2 .. w(n-1)] if ngram
    c_n : count of word 2 if bigrams or count of wn if ngram
    c12: count of bigram (w1, w2) if bigram or count of ngram if ngram
    N: the number of words in the corpus
    """

    p = c_n / N
    p1 = c_ngram / c_prior
    p2 = (c_n - c_ngram) / (N - c_prior)   
    # We proactively trap a runtimeWarning: divide by zero encountered in log,
    # which may occur with extreme collocations
    import warnings
    with warnings.catch_warnings(): # this will reset our filterwarnings setting
        warnings.filterwarnings('error')
        try:
            return (np.log(binom.pmf(c_ngram, c_prior, p)) 
                    + np.log(binom.pmf(c_n - c_ngram, N - c_prior, p)) 
                    - np.log(binom.pmf(c_ngram, c_prior, p1) )
                    - np.log(binom.pmf(c_n - c_ngram, N - c_prior, p2)))             
        except Warning:
            return np.inf 

In [387]:
len_prior = len(terms)
print("Au départ, on a {} ngrammes.".format(len_prior))

Au départ, on a 695 ngrammes.


In [388]:
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import binom, chi2
from nltk import bigrams, trigrams, ngrams

In [389]:
# Pour le calcul des probabilités, on a besoin de traiter séparément les ngrammes selon la valeur de n
N = len(tokens)
fd_ug = nltk.FreqDist(tokens)

bg = [tuple(tokenizer_re.tokenize(term[0])) for term in terms if len(tokenizer_re.tokenize(term[0])) == 2] # Bigrammes
fd_bg = nltk.FreqDist(bigrams(tokens)) # On calcule la distribution de fréquences à partir de tous les tokens pour éviter des divisions par zéro à l'étape suivante

tg = [tuple(tokenizer_re.tokenize(term[0])) for term in terms if len(tokenizer_re.tokenize(term[0])) == 3] # Trigrammes
fd_tg = nltk.FreqDist(trigrams(tokens)) # On calcule la distribution de fréquences à partir de tous les tokens pour éviter des divisions par zéro à l'étape suivante

qg = [tuple(tokenizer_re.tokenize(term[0])) for term in terms if len(tokenizer_re.tokenize(term[0])) == 4] # Quadgrammes
fd_qg = nltk.FreqDist(ngrams(tokens, n=4)) # On calcule la distribution de fréquences à partir de tous les tokens pour éviter des divisions par zéro à l'étape suivante

cg = [tuple(tokenizer_re.tokenize(term[0])) for term in terms if len(tokenizer_re.tokenize(term[0])) == 5] # 5-grammes
fd_cg = nltk.FreqDist(ngrams(tokens, n=5)) # On calcule la distribution de fréquences à partir de tous les tokens pour éviter des divisions par zéro à l'étape suivante

In [390]:
print("Voici quelques bigrammes : "+ str(random.sample(bg, 3)))
print("Voici quelques trigrammes : "+ str(random.sample(bg, 4)))
print("Voici quelques quadgrammes : "+ str(random.sample(bg, 2)))

Voici quelques bigrammes : [('professionnels', 'recherche'), ('expérience', 'clinique'), ('séance', 'spéciale')]
Voici quelques trigrammes : [('centre', 'éducatif'), ('psychiatric', 'emergency'), ('soutien', 'informationnel'), ('infirmier', 'ière')]
Voici quelques quadgrammes : [('psychothérapie', 'pqptm'), ('membre', 'indépendant')]


In [391]:
llr_bigrammes = []

for b in set(bg):
    c1 = fd_ug[b[0]]
    c2 = fd_ug[b[1]]
    c_ngram = fd_bg[b]

    res = -2 * loglikelihood_ratio(c1, c2, c_ngram, N)
    p = chi2.sf(res, 1) # 1 degrees of freedom
    if res == float('-inf') :
        res = np.inf

    if p < 0.05 or (res == float('inf')):
        llr_bigrammes.append({'Collocation' : " ".join(b).replace("' ", "'"), 'Fréquence' : c_ngram, 'LLR': res, 'p-value': p})

In [392]:
llr_trigrammes = []

for t in set(tg):
    c_prior = fd_bg[t[:2]] # Antécédent = P(w1w2) (si on considère que P (w1w2w3) = P(w3) | P(w1w2)
    c_n = fd_ug[t[2]]
    c_ngram = fd_tg[t] 

    res = -2 * loglikelihood_ratio(c_prior, c_n, c_ngram, N)
    p = chi2.sf(res, 1) # 1 degrees of freedom
    if res == float('-inf') :
        res = np.inf

    if p < 0.05 or (res == float('inf')):
        llr_trigrammes.append({'Collocation' : " ".join(t).replace("' ", "'"), 'Fréquence' : c_ngram, 'LLR': res, 'p-value': p})

In [393]:
llr_quadgrammes = []

for q in set(qg):
    c_prior = fd_tg[q[:3]] # Antécédent = P(w1w2w3) si on considère que P (w1w2w3w4) = P(w4 | P(w1w2w3)
    c_n = fd_ug[q[3]]
    c_ngram = fd_qg[q]

    res = -2 * loglikelihood_ratio(c_prior, c_n, c_ngram, N)
    p = chi2.sf(res, 1) # 1 degrees of freedom
    if res == float('-inf') :
        res = np.inf

    if p < 0.05 or (res == float('inf')):
        llr_quadgrammes.append({'Collocation' : " ".join(q).replace("' ", "'"), 'Fréquence' : c_ngram, 'LLR': res, 'p-value': p})

In [394]:
llr_5grammes = []

for q in set(cg):
    c_prior = fd_qg[q[:4]] # Antécédent = P(w1w2w3) si on considère que P (w1w2w3w4) = P(w4 | P(w1w2w3)
    c_n = fd_ug[q[4]]
    c_ngram = fd_cg[q]

    res = -2 * loglikelihood_ratio(c_prior, c_n, c_ngram, N)
    p = chi2.sf(res, 1) # 1 degrees of freedom
    if res == float('-inf') :
        res = np.inf

    if p < 0.05 or (res == float('inf')):
        llr_5grammes.append({'Collocation' : " ".join(q).replace("' ", "'"), 'Fréquence' : c_ngram, 'LLR': res, 'p-value': p})

In [395]:
df_bg = pd.DataFrame(llr_bigrammes)
df_tg = pd.DataFrame(llr_trigrammes)
df_qg = pd.DataFrame(llr_quadgrammes)
df_cg = pd.DataFrame(llr_5grammes)

df = df_bg.append(df_tg).append(df_qg).append(df_cg)
df.sort_values(['p-value'], 
            axis=0,
            ascending=[True], 
            inplace=True)

output_path = path.join('../04-filtrage/output/', acteur, acteur + '_significant-collocations.csv') 
df.to_csv(output_path)

C:\Users\p1115145\AppData\Local\Temp\ipykernel_13212\4089883408.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_bg.append(df_tg).append(df_qg).append(df_cg)
C:\Users\p1115145\AppData\Local\Temp\ipykernel_13212\4089883408.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_bg.append(df_tg).append(df_qg).append(df_cg)


### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Plan »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »
- « Personne »
- « Infirmière (en) »

In [396]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières
kw = ['programme', 'plan ', 'service', 'intervenant', 'infirmière en', 'institut', 'groupe de recherche', 'personne', 'maladie']

ngrammes_kwic = [" ".join([t[0] for t in ng]).replace("' ", "'") for ng in ngrammes]

In [397]:
extrant = pd.DataFrame(columns=['Mot-clé','Concordance', 'Fréquence'])
kwic = {w : [] for w in kw} 

In [398]:
for t in ngrammes_kwic: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
    for w in kw:
        if t.startswith(w):
            kwic[w].append(t)

In [399]:
kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [400]:
for term in kw:
    df = pd.DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence"])
    df.sort_values(["Fréquence"], 
        axis=0,
        ascending=[False], 
        inplace=True)

    df.insert(0, 'Mot-clé', term)
    extrant = pd.concat([extrant, df])


extrant = extrant[extrant['Fréquence'] > 30] 

file_path = '../04-filtrage/output/'
if sous_corpus:
    file_path = path.join(file_path, acteur, tag, tag)

else :
    file_path = path.join(file_path, acteur, acteur)


extrant.to_csv(file_path + '_KWIC' +'.csv')

### **Extraction de termes MeSH**

In [401]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/mesh-fr.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [402]:
extr_mesh = tokenizer_mesh.tokenize([t[0] for t in terms])

# **MODIF**

In [403]:
# MODIFICATION À APPORTER : dans l'extrant où on voit les termes mesh, ajouter la fréquence

termes_mesh[:15]

for t in termes_mesh:
    print(t, corpus.count(t))

psychiatrie légale 7247
soins aux patients 2052
santé mentale 1569
troubles mentaux 1144
réalité virtuelle 1085
développement durable 1066
formation en santé 1047
santé environnementale 1038
éthique de la recherche 1027
lignes directrices 1026
aspects légaux 1026
services sociaux 204
soins infirmiers 178
formation continue 92
formation continue 92
qualité de vie 90
curriculum vitae 81
curriculum vitae 81
minorités ethniques 70
facteurs de risque 64
projet de recherche 59
mots clés 56
meilleures pratiques 48
base de données 48
sars cov 48
agression sexuelle 41
gestion du risque 44
santé publique 41
trouble mental 39
services de santé 38
travail social 36
gestion des risques 35
évaluation éthique 34
congé de maladie 60
sciences humaines 32
affaires juridiques 28
psychologie clinique 70
troubles mentaux graves 28
travailleurs sociaux 27
sécurité civile 26
professionnels de la santé 25
gestion des ressources humaines 25
gestion du personnel 24
recherche en soins infirmiers 24
examen éthiqu

In [404]:
termes_mesh = []

for t in extr_mesh:
    if t in mesh:
        termes_mesh.append(t)

In [ ]:
file_path = '../04-filtrage/output/'
if sous_corpus:
    file_path = path.join(file_path, acteur, tag, tag)

else :
    file_path = path.join(file_path, acteur, acteur)

df = DataFrame(termes_mesh)
df.to_csv(file_path + '_MeSH.csv')

### **Extraction de termes SNOMED**

In [ ]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/SNOMED_fr.csv'

with open(file_path, 'r', encoding='utf-8') as f:
    sm = read_csv(f, sep=';')
    sm = list(dict.fromkeys([str(t).strip().lower() for t in sm['term'].tolist()]))

    sm = [tuple(tokenizer_re.tokenize(w)) for w in sm if len(w.split()) > 1]
    tokenizer_sm = MWETokenizer(sm, separator = ' ')

    sm = [tokenizer_sm.tokenize(w)[0].lower() for w in sm]

In [ ]:
extr_sm = tokenizer_sm.tokenize([t[0] for t in terms])

In [ ]:
termes_sm = []

for t in extr_sm:
    if t in sm:
        termes_sm.append(t)

In [ ]:
file_path = '../04-filtrage/output' 
if sous_corpus:
    file_path = path.join(file_path, acteur, tag, tag) 

else :
    file_path = path.join(file_path, acteur, acteur)


df = DataFrame(termes_sm)

df.to_csv(file_path + '_SNOMED.csv')